In [1]:
import pandas as pd
import re
import numpy as np
import os

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [63]:
'''
Import our main reference file
'''
main_df = pd.read_csv('C:/Users/Desktop/Work/Platform Migration/Migration Tracker/111623_Reference Split.csv', low_memory = False)

In [50]:
'''
First, we need to identify and handle NA values in our df.
'''
# Identify rows where 'reference_id' is not numeric
non_numeric_rows = main_df[pd.to_numeric(main_df['reference_id'], errors='coerce').isna()]

# Display the rows with non-numeric 'reference_id'
print(non_numeric_rows)


Empty DataFrame
Columns: [id, name, reference_id, accountno, reference]
Index: []


In [64]:
'''
After identifying the problematic rows, tt looks like the main issue is they're just NaN codes. 
Because there is no corresponding value in the reference field, we can simply drop them from our process.

Finally, we truncate the floating decimals by converting the field into an integer.
'''

main_df = main_df.dropna(subset=['reference_id'])
main_df['reference_id'] = main_df['reference_id'].astype(int)
main_df['accountno'] = main_df['accountno'].astype(int)

In [38]:
'''
As specified by the dev team, we need to make sure the character level isn't greater than 50 and 255, respectively, for ref_id and reference content.
'''

max_length = main_df['reference'].astype(str).apply(len).max()
print(max_length)

40


In [66]:
# First, I'd like to create a unique mapping list for later reference.

# Create a new DataFrame with unique combinations
unique_df = main_df.drop_duplicates(subset=['accountno', 'name', 'id']).copy()
unique_df.drop(columns=['reference_id', 'reference'], inplace=True)

# Sort the DataFrame based on the 'id' field
unique_df_sorted = unique_df.sort_values(by='id')

In [67]:
'''
Rename the columns to the format specified by engineering.
'''

main_df.rename(columns={'reference_id': 'Reference_name', 'reference': 'Reference_description'}, inplace=True)

In [ ]:
# Create a copy of main_df with the dropped columns #
drop_df = main_df.drop(columns=['name', 'accountno'])
drop_df['Reference_name'] = drop_df['Reference_description']
drop_df['Reference_description'] = ''

In [ ]:
'''
The max lengths are 5 and 40, respectively. I think now we can 1) drop fields we don't need and 2) split out the parent file into child files for 
Engineering to process.
'''

# Write the function to split and save the CSVs #
def split_and_save_csv_by_portal_id(df, output_directory):
    
    # Create a directory to save the CSV files if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Get unique portal IDs
    unique_portal_id = drop_df['id'].unique()

    # Iterate through unique portal IDs and save corresponding CSV files
    for portal_id in unique_portal_id:
        # Create a DataFrame with records matching the current portal ID
        filtered_df = drop_df[drop_df['id'] == portal_id]
        
        # Generate the CSV file name based on the portal ID; use [0] value access to grab just the name     
        csv_filename = os.path.join(output_directory, f"cid_reference_{portal_id}.csv")

        
        # Save the filtered DataFrame to a CSV file
        filtered_df.to_csv(csv_filename, index=False)
        print(f"Saved CSV for client {portal_id} to {csv_filename}")

# Usage example
output_dir = 'C:/Users/Desktop/Work/Platform Migration/Data Cleaning/Reference CSVs'  # Specify the output directory
split_and_save_csv_by_portal_id(main_df, output_dir)

In [80]:
'''
Now we just need to drop the unneeded columns from our drop_df file.
'''

directory = 'C:/Users/Desktop/Work/Platform Migration/Data Cleaning/Reference CSVs'

csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Iterate through each x in our y (each CSV file in our list); explicitly open each file, select only every column past the first two, and then save 
# them to the original file location.
for file in csv_files:
    file_path = os.path.join(directory, file)
    csv_df = pd.read_csv(file_path)
    csv_df = csv_df.drop(['id'], axis=1)
    csv_df.to_csv(file_path, index = False)